In [1]:
from NeuralNetwork import NeuralNetwork, FC, VGG, VGG16, EfficientNetB0
from Dataset import Dataset, SPIRAL, MOON, CIRCLE, MNIST, Fashion_MNIST, CIFAR10
from NeuroEvolution import NeuroEvolution
from tensorflow import keras
from Representation import Function, PAU
from Activation import PANGAEA_Activation, Pade_Activation_Unit
import tensorflow as tf
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from numpy import random
from Swarm import Star, Swarm
import numpy as np
import copy 

np.set_printoptions(suppress=True)

# Dataset

In [2]:
dataset = CIFAR10
dataset_train = dataset(training_split=.8)


# Neural Network

In [3]:
blocks = 3
nn = VGG(dataset_train, blocks)

# Randomness Settings

In [4]:
metric = keras.metrics.CategoricalAccuracy()
loss_noise = 0.
activation_noise = 0.
input_noise = 0.
gradient_noise =  0.
weight_noise = 6
label_smoothing = 0.0
gradient_dropout = 0.
dropout = 0.0
drop_connect = 0.
batch_size = 128
batch_schedule = 0.
drnn = 0.
weight_std =  0.05
verbose = 1
epochs = 20
iterations = 100000000
patience = np.inf
save_best = False
sleep = 0.
cut_threshold = 0.12
batch_range = [128, 1024]
random_flip=1
random_rotation=0.0
random_zoom=0.0
random_translation=0.0
random_contrast=0.
shuffle = 0
lr = 3
optimizer = 1
lr_schedule = 0.0
lr_range = [1, 5]



nn.set_config(shuffle, random_flip, random_rotation, random_zoom, random_translation, random_contrast, input_noise,
              label_smoothing, weight_std, dropout, drop_connect, drnn, activation_noise, loss_noise,
              optimizer, lr, lr_schedule, batch_size, batch_schedule, weight_noise, gradient_noise, gradient_dropout,              
              metric, epochs, iterations, patience, verbose=verbose,
              batch_range=batch_range, lr_range=lr_range, sleep=sleep, save_best=save_best, cut_threshold=cut_threshold)
nn.create_model()


In [5]:
nn.model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 gaussian_noise (GaussianNoi  (None, 32, 32, 3)        0         
 se)                                                             
                                                                 
 drop_connect (DropConnect)  (None, 32, 32, 32)        896       
                                                                 
 noisy_re_lu (NoisyReLU)     (None, 32, 32, 32)        0         
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 drop_connect_1 (DropConnect  (None, 32, 32, 32)       9248  

In [6]:
methods = ['shuffle', 'flip', 'rotation', 'zoom', 'translation', 'contrast'
                               , 'input noise' , 'label smoothing' , 'weight init', 'dropout', 'dropconnect',
                               'drnn', 'activation noise', 'loss noise', 
                               'optimizer', 'lr', 'lr schedule', 'batch', 'batch schedule', 'weight noise', 'gradient noise', 
                               'gradient dropout']

default_settings = [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 3, 0.0, 128, 0.0, 6.0, 0.0, 0.0]

# Training with default randomness settings

In [7]:
fitness = NeuroEvolution.evaluate(default_settings, [nn], test_time=True, verbose=1)[0]     


loss_noise: 0.0 , activation_noise: 0.0 , input_noise: 0.0 , label_smoothing: 0.0 , weight_noise: 0 , gradient_dropout: 0.0 , gradient_noise: 0.0 , batch_size: 128 , dropout: 0.0 , drop_connect: 0.0 , batch_schedule: 21 , drnn: 0.0 , weight_std: 0.05 , flip: 1 , rotation: 0.0 , zoom: 0.0 , translation: 0.0 , contrast: 0.0 , shuffle: 0 , lr: 0.001 , optimizer: 1 , lr_schedule: 21 , batch_increase: 0 , lr_increase: 0

Start of epoch 0
cuurent batch: 128
Training acc: 0.4495
Validation acc: 0.4573
Time taken: 14.24s
Patience:  1

Start of epoch 1
cuurent batch: 128
Training acc: 0.6160
Validation acc: 0.5648
Time taken: 8.63s
Patience:  1

Start of epoch 2
cuurent batch: 128
Training acc: 0.7072
Validation acc: 0.6438
Time taken: 8.92s
Patience:  1

Start of epoch 3
cuurent batch: 128
Training acc: 0.7706
Validation acc: 0.6607
Time taken: 8.64s
Patience:  1

Start of epoch 4
cuurent batch: 128
Training acc: 0.8220
Validation acc: 0.6926
Time taken: 8.42s
Patience:  1

Start of epoch 5
c

In [8]:
print("Test Accuracy:", fitness)

Test Accuracy: 0.7531999945640564


# PSO search

In [23]:
population_size = 10
ind_size = 22
options = {'c1': 1.49618, 'c2': 1.49618, 'w': 0.7298, 'k': population_size, 'p': 2}
bounds = [[0.0, 1.], [0.0, 4.], [0.0, 0.2], [0.0, 0.2],  
          [0.0, 0.2], [0.0, 0.2], [0.0, 1.], [0.0, 1.], 
          [0.0, 0.1], [0.0, 0.5], [0.0, 0.5], [0.0, 1.], 
          [0.0, 0.1], [0.0, 1.], [0.0, 3.0], nn.lr_range, 
          [-1.0, 1.0], nn.batch_range, [-1.0, 1.0], 
          [1.0, 6.0], [0.0, 1.0], [0.0, 0.50]]


star = Star(population_size)
swarm = Swarm(star, population_size, ind_size, options, bounds, 0.2, bounded=True)
for i in range(population_size):
        choice = np.random.choice([1,2,3,4,5,6,7, 9, 10, 11, 12, 13, 19,20,21], 4, replace=False)
        for j in range(ind_size):
            if j not in choice and j not in [0, 8, 14, 15, 16, 17, 18]:
                swarm.position[i][j] = default_settings[j]

ost, pos, time_found = swarm.optimize(nn, steps=200, no_change=5, verbose=1)

In [9]:
fitness = NeuroEvolution.evaluate(pos, [nn], test_time=True)[0]


loss_noise: 0 , activation_noise: 0 , input_noise: 0 , label_smoothing: 0 , weight_noise: 0 , gradient_dropout: 0.102167731 , gradient_noise: 0 , batch_size: 128 , dropout: 0 , drop_connect: 0 , batch_schedule: 20 , drnn: 0 , weight_std: 0.010547912 , flip: 3 , rotation: 0 , zoom: 0 , translation: 0.102922781 , contrast: 0.133724907 , shuffle: 1 , lr: 0.0010899168831738897 , optimizer: 1 , lr_schedule: 18 , batch_increase: 0 , lr_increase: 0

Start of epoch 0
cuurent batch: 128
Training acc: 0.4854
Validation acc: 0.3365
Time taken: 10.54s
Patience:  1

Start of epoch 1
cuurent batch: 128
Training acc: 0.5996
Validation acc: 0.6611
Time taken: 9.36s
Patience:  1

Start of epoch 2
cuurent batch: 128
Training acc: 0.6851
Validation acc: 0.7066
Time taken: 9.14s
Patience:  1

Start of epoch 3
cuurent batch: 128
Training acc: 0.7309
Validation acc: 0.7290
Time taken: 9.91s
Patience:  1

Start of epoch 4
cuurent batch: 128
Training acc: 0.7640
Validation acc: 0.7594
Time taken: 9.47s
Patie

In [10]:
print("Test Accuracy:", fitness)

Test Accuracy: 0.843999981880188
